In [4]:
from fastai.vision import *
import os 
import cv2
import json

In [ ]:
path = Path('./')
path.ls()

In [ ]:
src_data_path='../data/jinnan2_round1_train_20190305/'
annotations_images_path='../data/annotations_images/'


In [ ]:
if not os.path.exists('../data/train/'):
        os.makedirs('../data/train/')
        
if not os.path.exists('../data/train/normal/'):
        os.makedirs('../data/train/normal/')
        
if not os.path.exists('../data/train/restricted/'):
        os.makedirs('../data/train/restricted/')
        
if not os.path.exists(annotations_images_path):
        os.makedirs(annotations_images_path)
        

In [ ]:
with open(src_data_path + 'train_no_poly.json', 'r') as f:
    load_dict = json.load(f)
   

info = load_dict['info']
licenses = load_dict['licenses']
categories = load_dict['categories']
images = load_dict['images']
annotations = load_dict['annotations']

print(info)

In [ ]:
imageid2name = {}
for i, image in enumerate(images):
    imageid2name[image['id']] = image['file_name']

In [ ]:
def overlay_class_box(image, annotations):
        for i, annotation in enumerate(annotations):
            category_id = annotation['category_id']
            box = annotation['bbox']
            minAreaRect = annotation['minAreaRect']
        
            xmin, ymin = box[:2]
            xmax, ymax = box[0]+box[2], box[1]+box[3]
            cv2.putText(
                image, str(category_id), (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 125), 2 
            )#(B,G,R)
        
            image = cv2.rectangle(
                image, (xmin, ymin), (xmax, ymax), (0,0,255), 2 
            )

        return image

In [ ]:
img_dict = {}
for i, anno in enumerate(annotations):
    image_id = anno['image_id']
    item = {
            'category_id': anno['category_id'],
            'bbox': anno['bbox'],
            'minAreaRect': anno['minAreaRect']
        }
    
    if image_id in img_dict.keys():
        img_dict[image_id].append(item)
    else:
        img_dict[image_id] = [item]

创建在图片上画上标签矩形的数据集

In [ ]:
for i, image_id in enumerate(img_dict.keys()):
    file_name = imageid2name[image_id]
    img = cv2.imread(src_data_path + 'restricted/' + file_name)
    img = overlay_class_box(img, img_dict[image_id])
    if i < 2:
        plt.imshow(img)
        plt.show()
    
    cv2.imwrite(annotations_images_path  + file_name, img)
    #print('save {} img to restricted_with_annotations dir'.format(file_name))
    
print('process success!!!!')
print(i)

In [5]:
def copy_files(src_path, dest_path, flag_str):
    files = os.listdir(src_path)
    print('test img nums={}'.format(len(files)))
    for file in files:                
       
        
        full_src_path = src_path+file;
        full_dest_path=dest_path+flag_str+file
        
               
        cmd = 'cp '+full_src_path+' '+full_dest_path+' -rf'
        tmp = os.popen(cmd).readlines()

In [ ]:
copy_files(annotations_images_path,'../data/train/restricted/','annotations_' )

In [ ]:

cmd = 'cp '+src_data_path+'restricted/*'+' '+'../data/train/restricted/'+' -rf'
tmp = os.popen(cmd).readlines()
    
cmd = 'cp '+src_data_path+'normal/*'+' '+'../data/train/normal/'+' -rf'
tmp = os.popen(cmd).readlines()  



In [ ]:
cmd = 'rm '+annotations_images_path +' -rf'
tmp = os.popen(cmd).readlines() 
    

In [6]:

#copy_files('../data/train/restricted-back/','../data/train/restricted/','copy_' )

test img nums=1953
